In [1]:
# Model training for DRMM
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.regularizers import l1, activity_l1, l2
from keras.layers import Input, LSTM, Dense, merge, Dropout
from keras.models import Model
from collections import defaultdict
import numpy as np
import make_test_data
import pickle as pkl
import os
import pdb
from sklearn.linear_model import LogisticRegression

#TODO: fix root, remove key restrictions

# Placeholder data parsing
def get_data(histograms, training_year=2014, training=True):
    # Proposed format: [doc_id topic_id h0 h1 h2 h3 ...]
    #data_root = '/scratch/cluster/dnelson/ir_proj'

    # can set this to 2015 to read in annotations for 2015 instead
    # format: label_dict[doc_id][topic_id] = ground truth
    label_dict = make_test_data.make_truth(training_year)

    # enforcing order on a dictionary & downsampling to only data w/judgments for training
    key_array = histograms.keys()
    if training:
        key_array = [val for val in key_array if label_dict[int(val[0])][int(val[1])+1] >= 0]

    X = np.array([[hist_list[1] for hist_list in histograms[key]] for key in key_array])
    Y = np.array([label_dict[int(key[0])][int(key[1])+1] for key in key_array])
    
    MAX_LEN = 25
    X = [val + [np.zeros(29)]*(MAX_LEN-len(val)) if len(val) < 25 else val[:25] for val in X]
    return X, Y, key_array

def get_dict(training_year=2014):
    data_root = '/Users/Dan/class/deep_ir/project/data'
    
    with open(os.path.join(data_root, 'term_histograms_%d' % training_year), 'r') as f:
        histograms = pkl.load(f)
    return histograms


def get_fake_data():
    X_train = np.random.rand(1000, 29)

    # roughly uniform dist of 0, 0.5, 1
    rand_vals = np.random.rand(1000)
    Y_train = np.array([np.floor(val * 3)/2.0 for val in rand_vals])
    return X_train, Y_train

Using TensorFlow backend.


In [2]:
# Get all data for maximum iteration funtime power
# THIS IS THE LONG PART
train_dict = get_dict(training_year=2014)
test_dict = get_dict(training_year=2015)

In [28]:
X_train, Y_train, _ = get_data(train_dict, training_year=2014, training=True)


qrels2014.txt


In [27]:
#X_train, Y_train = get_fake_data()
X_test, Y_test, test_keys = get_data(test_dict, training_year=2015, training=False)

qrels-treceval-2015.txt


In [57]:
# ASIDE - this is how you make your own loss function. Not necessary but fun to play with.
from keras import backend as K

def supercool_loss(y_true, y_pred):
    return K.mean(K.abs(y_true*10 - y_pred))
    

In [2]:
from keras.regularizers import l1, activity_l1, l2
from keras.layers import Input, LSTM, Dense, merge, Dropout
from keras.models import Model

In [3]:
# Create model (input_shape is inferred after first layer)
# This model is a regression

# Defines all shared weight layers for feedforward network
num_inputs = 25
shared_dense = Dense(5, activation='relu')
shared_dropout = Dropout(0.5)
shared_score = Dense(1, activation='relu')

# constructs parallel shared layers for each input
inputs = [Input(shape=(29,)) for val in range(num_inputs)]
output_1 = [shared_dense(val) for val in inputs]
output_d = [shared_dropout(val) for val in output_1]
output_score = [shared_score(val) for val in output_d]
concat_vals = merge(output_score, mode='concat')

# Defines term weighting network
shared_term_weight = Dense(1, activation='relu')
term_weight_list = [shared_term_weight(val) for val in inputs]
term_weights = merge(term_weight_list, mode='concat')

# Dot product over terms to weight them properly
output = merge([concat_vals, term_weights], mode='dot', dot_axes=1)
final_output = Dense(1, activation='relu')(output)

model = Model(input=inputs, output=final_output)
# Compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=1.)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['binary_accuracy'])

In [25]:
# X_train needs to be a list of numpy arrays
X_train = np.array(X_train)
X_train_list = [X_train[:,val,:] for val in range(X_train.shape[1])]
model.fit(X_train_list, Y_train, nb_epoch=1, batch_size=20)

Epoch 1/1


InvalidArgumentError: Matrix size-compatible: In[0]: [20,20], In[1]: [1,1]
	 [[Node: MatMul_225 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/cpu:0"](BatchMatMul_2, dense_12_W/read)]]
Caused by op u'MatMul_225', defined at:
  File "/Users/Dan/anaconda/envs/hw2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/Dan/anaconda/envs/hw2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/Dan/anaconda/envs/hw2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/Dan/anaconda/envs/hw2/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/Users/Dan/anaconda/envs/hw2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/Dan/anaconda/envs/hw2/lib/pyt

In [20]:
new_train_list = [val[:10] for val in X_train_list]
test_preds = model.predict(new_train_list)

In [5]:
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 29)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 29)            0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 29)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 29)            0                                            
___________________________________________________________________________________________

In [29]:
print Y_train[:100]

[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.
  0.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   1.
  0.   1.   0.   0.   0.   0.   0.   0.   0.   0.5  0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.5  1.   0.   0.   0.   0.   0.   0.   0. ]


In [98]:
# Test model
X_test = np.array(X_test)
X_test_list = [X_test[:,val,:] for val in range(X_test.shape[1])]

pred_ranks = model.predict(X_test_list, batch_size=491)

In [46]:


#[0][0] = id, [0][1] = topic, [1] = rank
ranks_and_keys = zip(test_keys, pred_ranks)
result_dict = defaultdict(lambda: [])
for val in ranks_and_keys:
    result_dict[val[0][1]].append((val[1][0], val[0][0]))

with open('cat_query_results','wb') as f:
    for cur_topic in range(30):
        topic_results = result_dict[str(cur_topic)]
        topic_results.sort(reverse=True)

        for result_rank, cur_result in enumerate(topic_results[:1000]):
            line_to_write = [str(cur_topic + 1), '0', str(cur_result[1]), str(result_rank + 1), str(cur_result[0]), 'test_run', '\n']
            f.write(" ".join(line_to_write))

In [88]:
lens = [len(val) for val in X_test]
from collections import Counter
counts = Counter(lens)
print counts.most_common()

[(25, 37807)]


In [103]:
print 

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   5.  11.  33.
   39.  25.  15.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4.  28.
   47.  36.   7.   6.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  28.  36.
   39.  19.   4.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   5.   1.   5.  14.
   28.  33.  22.  15.   5.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   9.  16.
   20.  50.  24.   7.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   5.  18.  35.  26.
   24.   7.   5.   5.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   8.   9.
   36.  29.  38.

In [15]:
print train_dict.keys()[:10]

[('2794284', '20'), ('3510867', '23'), ('2275225', '21'), ('3658214', '26'), ('2830982', '4'), ('2533397', '5'), ('3838404', '12'), ('1570135', '7'), ('3565924', '6'), ('3274658', '12')]


In [16]:
for val in train_dict.keys()[:10]:
    print train_dict[val]

[  0   0   0   0   0   0   0   0   0   0   1  10  93 301 428 335  90  39
   9   5   4   0   0   0   0   0   0   0   1]
[  0   0   0   0   0   0   0   0   0   0   4  27 189 504 524 399 188  85
  35  25   5   1   0   0   1   0   0   0   2]
[  0   0   0   0   0   0   0   0   0   0   2  20  84  96 111  62  25  13
   2   0   0   0   0   0   0   0   0   0   0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  9 57 82 56 34  8  8  4  1  0  0  0  0
  0  0  0  2]
[  0   0   0   0   0   0   0   0   0   0   1  35 150 406 558 502 233 134
  54  13   4   3   0   2   1   1   0   0   2]
[  0   0   0   0   0   0   0   0   0   0   3  27 110 245 226 179  82  58
   8  10   3   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   3  34 117 232 273 135  86  42
   6  12   1   0   0   1   1   1   0   0   1]
[  0   0   0   0   0   0   0   0   0   0  16  75 258 451 560 394 219  84
  54   2   2   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   2   8  66 371 435 424 195  75
  60

In [18]:
thing = [[val, val, val] for val in X_train]
print np.array(thing).shape

(31403, 3, 29)


In [74]:
print len(X_train[0])

25


In [83]:
print np.array(X_train).shape[1]

25


In [13]:
output_1 = [shared_dense(val) for val in inputs]

In [14]:
print output_1

[<tf.Tensor 'Relu_2:0' shape=(?, 5) dtype=float32>, <tf.Tensor 'Relu_3:0' shape=(?, 5) dtype=float32>, <tf.Tensor 'Relu_4:0' shape=(?, 5) dtype=float32>]
